# Segmenting and Clustering Neighborhoods in Toronto

In [79]:
import numpy as np 
import pandas as pd 

In [60]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executi

In [61]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 


In [64]:
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

In [66]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


### The table has only three columns

In [68]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_parsed_result = BeautifulSoup(requests.get(url).text, 'html.parser')
nb_t= html_parsed_result.find('table', class_ = 'wikitable')
nb_r= nb_t.find_all('tr')
nb_info = []

In [69]:
for row in nb_r:
    info = row.text.split('\n')[1:-1] 
    nb_info.append(info)    
nb_info[0][-1] = 'Neighborhood'
nb_df = pd.DataFrame(nb_info[1:], columns=nb_info[0])

In [70]:
nb_df = pd.DataFrame(nb_info[1:], columns=nb_info[0])

In [71]:
nb_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']
for i in nb_not_assigned:
    nb_df['Neighborhood'][i] = nb_df['Borough'][i]

In [72]:
boroughs_not_assigned = nb_df.index[nb_df['Borough'] == 'Not assigned']
neighborhoods_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']

### Only process the cells that have an assigned borough. Ignore cells with a borough that = Not assigned.

In [73]:
neighborhoods_borough_not_assigned = boroughs_not_assigned & neighborhoods_not_assigned
nb_df.drop(nb_df.index[boroughs_not_assigned], inplace=True)
nb_df.reset_index(drop=True, inplace=True)
nb_df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [75]:
gp = nb_df.groupby('Postcode')
gp_nb = gp['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
gp_boroughs = gp['Borough'].apply(lambda x: set(x).pop())
gp_df = pd.DataFrame(list(zip(gp_boroughs.index, gp_boroughs, gp_nb)))
gp_df.columns = ['Postcode', 'Borough', 'Neighborhood']

In [76]:
gp_df.head(12)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [77]:
print('The DataFrame shape is', gp_df.shape)

The DataFrame shape is (103, 3)


#### Map generation.

In [80]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [81]:
import geocoder 


In [82]:
# The code was removed by Watson Studio for sharing.

In [83]:
# The code was removed by Watson Studio for sharing.

In [84]:
# The code was removed by Watson Studio for sharing.

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [85]:
toronto_df.to_csv('Segmenting_Clustering_Neighborhoods_Toronto_final.csv',index=False)

In [88]:
map = folium.Map(location=[43.6532,-79.3832], zoom_start=11)
data_csv = "Segmenting_Clustering_Neighborhoods_Toronto_final.csv"
coordinate_df = pd.read_csv(data_csv)
for location in coordinate_df.itertuples():
    label = 'Postal Code: {};  Borough: {};  Neighborhoods: {}'.format(location[1], location[2], location[3])
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [location[-2], location[-1]],
        radius=1,
        color='purple',
        fill=True,
        fill_color='#e67777',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
    folium.Circle(
        radius=500,
        popup=label,
        location=[location[-2], location[-1]],
        color='#e67777',
        fill=True,
        fill_color='#e67777'
    ).add_to(map) 

In [89]:
map

In [90]:
print('The DataFrame shape is', gp_df.shape)

The DataFrame shape is (103, 3)
